# Processing California's public payrolls

This notebook processess government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [ ]:
import pandas as pd
import zipfile
from urllib.request import urlopen 
import pyarrow
import os
import cpi
import glob
import requests
import matplotlib
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Download zipped salary tables by year and agency type

In [ ]:
# cpi.update()

In [ ]:
os.chdir('/Users/mhustiles/data/data/controller/input/')

In [ ]:
formaturl = lambda x: 'https://publicpay.ca.gov/RawExport/' + f'{x[1]}_' + f'{x[0]}' + '.zip'

In [ ]:
metadata = []
for y in range(2009,2021):
    for e in ['City', 'County', 'SpecialDistrict', 'StateDepartment']:
#     for e in ['City', 'County']:
        metadata.append(dict(entity = e, year = y, url = formaturl((e, y))))

### Extract CSVs from .zip files, and then discard the .zip files

In [ ]:
for m in metadata:
    !wget '{m['url']}'
    !unzip \*.zip
    !rm -f *.zip

---

### Read all the text files, loop and store them in a dataframe

In [ ]:
path = '/Users/mhustiles/data/data/controller/input/'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, encoding = "ISO-8859-1", \
            low_memory=False, dtype = {'DepartmentOrSubdivision': 'object', 'Year': 'object'}) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

### Trim the dataframe to the columns we need

In [ ]:
payroll = \
pd.DataFrame(concatenated_df[['Year','EmployerType','EmployerPopulation','EmployerName','DepartmentOrSubdivision',\
                 'Position','OvertimePay','TotalWages', 'TotalRetirementAndHealthContribution', 'EmployerCounty']])

### Not everyone reports (or pays) overtime

In [ ]:
payroll['OvertimePay'].fillna(0, inplace = True)
payroll['DepartmentOrSubdivision'].fillna('NOT LISTED', inplace = True)
payroll['EmployerPopulation'].fillna(0, inplace = True)

### Clean up column headers

In [ ]:
payroll.columns = payroll.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [ ]:
payroll.rename(columns = {
'employertype':'type',
'employerpopulation':'population',
'employername':'employer',
'departmentorsubdivision':'department',
'overtimepay':'overtime',
'totalretirementandhealthcontribution':'benefits',
'totalwages':'wages',
 }, inplace = True)

### Uppercase everything because their title casing across hundreds of agencies is janky

In [ ]:
payroll = payroll.apply(lambda x: x.astype(str).str.upper())

### Convert year column to datetime year for CPI function

In [ ]:
payroll['year'] = pd.to_datetime(payroll['year'])

In [ ]:
payroll['year'] = payroll['year'].dt.year

### Adjust overtime on each row for CPI according to its respective year

In [ ]:
payroll[['overtime', 'population', 'wages', 'benefits']] = \
payroll[['overtime', 'population', 'wages', 'benefits']].astype(float)

In [ ]:
payroll['basewages'] = (payroll['wages'] - payroll['overtime']).astype(float)

In [ ]:
payroll['adjusted_overtime'] = payroll.apply(lambda x: cpi.inflate(x.overtime, x.year), axis=1)

In [ ]:
payroll['adjusted_wages'] = payroll.apply(lambda x: cpi.inflate(x.wages, x.year), axis=1)

In [ ]:
payroll['adjusted_benefits'] = payroll.apply(lambda x: cpi.inflate(x.benefits, x.year), axis=1)

In [ ]:
payroll['adjusted_basewages'] = payroll.apply(lambda x: cpi.inflate(x.basewages, x.year), axis=1)

### How does the dataframe look? 

In [ ]:
payroll.head()

### How many records do we have here?

In [ ]:
# How many records?
len(payroll)

### Clean up some employer names

In [ ]:
payroll['employer']\
    .replace('SACRAMENTO METROPOLITAN FIRE DISTRICT', 'SACRAMENTO METRO FIRE', inplace=True)

In [ ]:
payroll['employer']\
    .replace('FORESTRY & FIRE PROTECTION, CALIFORNIA DEPARTMENT OF', 'CALFIRE', inplace=True)

In [ ]:
payroll['type']\
    .replace('STATE DEPARTMENT', 'STATE', inplace=True)

In [ ]:
payroll['employer']\
    .replace('ORANGE COUNTY FIRE AUTHORITY', 'OC FIRE AUTHORITY', inplace=True)

### Export table of all titles to help ID fire roles

In [ ]:
positions_all = payroll.groupby(['position', 'employer', 'type', 'department'])\
    .agg('size').reset_index()

In [ ]:
positions_all['position'] = positions_all['position']\
                            .str.replace('!ST ', '', regex=False)\
                            .str.replace('1ST ', '', regex=False)\
                            .str.replace('2ND', '', regex=False)\
                            .str.replace('.E.', '', regex=False)\
                            .str.replace(' - ', '', regex=False)\
                            .str.replace('\d', '')\
                            .str.replace('[.!?\\-]', '')

In [ ]:
positions_fire = positions_all[ (positions_all['position'].str.contains('FIRE')) |\
                                       (positions_all['type'].str.contains('FIRE')) |\
                                       (positions_all['department'].str.contains('FIRE')) |\
                                       (positions_all['employer'].str.contains('FIRE'))]

In [ ]:
positions_fire_grouped = positions_fire.groupby(['position']).agg('size')

In [ ]:
positions_fire_grouped.to_csv('/Users/mhustiles/data/github/notebooks/california-fire-payroll/input/positions_fire_grouped.csv', header=True)

---

### Export to a lightweight format

In [ ]:
payroll.reset_index().to_feather('/Users/mhustiles/data/data/controller/output/payroll.feather')

In [ ]:
payroll.reset_index().to_csv('/Users/mhustiles/data/data/controller/output/payroll.csv')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx